# Asynchronous programming

- an asynchronous operation is one that executes in the background
- it’s not always the most appropriate solution

## What’s the Deal with Asynchrony?

- CPUs spend most of their time waiting for I/O operations

### Synchronous Interfaces

- execute only a single operation at a time
- hide the I/O waiting (referred to as _blocking_)
- easy to work with and simple to reason about

### Multithreading

- each thread is responsible for executing a particular independent sequence of blocking operations
- threads communicate with each other using a synchronization primitives like locks, channels
- drawbacks:
  - additional complexity due to threads coordination
  - context-switching cost
- concurrency: the ability to have multiple independent operations that can run interleaved
- parallelism: multiple tasks are executing at the same time
  - added complexity to handle truly simultaneous access to shared data structures

### Asynchronous Interfaces

- _nonblocking_ interfaces: the result will be available at some later time
- In Rust, is a method that returns a `Poll`

### Standardized Polling

- to avoid proliferation of polling methods, implement the `Future` trait
- Types that implement the `Future` trait are known as _futures_ and represent values that will be eventually available in the future
- In general, you should not poll a future again after it has returned `Poll::Ready`

### Ergonomic Futures

- implementing `Future` for a type is fairly complex, and it is rarely done
- there are more ergonomic APIs to implement _nonblocking_ interfaces

#### async/await

- these primitives rely on _generators_
- a _generator_ is a piece of code that can be stopped and resumed
- when stopped, it saves the state into a compiler-generated data structure
- _yielding_: a resumable return
- in Rust, at the moment, generators are only used internally???
- since generators need to keep state, and they can be nested, they can increase the memory footprint a lot
  - this can impact runtime performance due to copy

#### Pin and Unpin

- Future types are self-referential
- a self-referential type can't be moved safely (moving doesn't update the pointers, so they'll still be pointing at the old memory address)
- Pin wraps a pointer and stops its value from moving (unless the underlying type implements `Unpin`)

### Going to Sleep

- _executor_: take care of checking if a future can be pulled again

#### Waking Up

- goal is to poll only when the future can actually make progress
- the executor create a `Waker` should be called when the future can again make progress
- the `Waker` is passed as a callback to the future

#### Fulfilling the Poll Contract

- it is the future’s responsibility to ensure that something calls wake on the provided `Waker` when the future is next able to make progress.
- the leaf futures actually interact with wakers
  - the futures that wait for events that originate within the same process can simply call `wake` then the events happen
  - the futures waiting for external events provided by the executor crate
    - the futures tell the executor which external events they are waiting for within a `Waker`
    - the executor ask the operating system if any of the requested resources have new events
    - the executor calls `wake` on the corresponding wakers

#### Waking Is a Misnomer

- `Waker::wake` actually signals that a particular future is _runnable_
- lets the executor
  - known when it can sleep
  - avoid polling futures unnecessarily

#### Tasks and Subexecutors

- The futures in an asynchronous program form a tree:
- the roots are called _tasks_
- the roots are given to the executor
- the executor polls the roots
- each future have to choose which of its children future pool, thus behaving as a _subexecutors_

## Tying It All Together with spawn

- a server can be implemented as:
  - a future that listens for new connections
  - for each incoming connection it spawns a new future to handle it
  - such new future is handled by the executor and will run concurrently
